![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_5 Graphs and gauges.

Using what we have learned so far, let's add some graphs to the display. This way we can see a record over time of the temperature and humidity readings. 

We use Plotly to get some nice looking gauges, and add some graphs to record the data coming from the Phidget devices. 

It is important to run this notebook in the Google Chrome web browser, on a desktop or laptop computer. It also will work with Chromium, Opera and Edge. Unfortunately, other browsers (FireFox, Safari) will not work as they cannot access the USB port that connects to the Phidget device. That may change in the future, but at the time of writing, this is currently only possible in Chrome/Chromium/Opera/Edge.

## Setting up the Phidget hardware

We will be using the Phidget Starter Kit, as shown in this figure:
<img src="images/KIT4003.jpg" alt="Image of the starter kit"  width = 500 />

Verify these points:
- Get a hold of a Phidgets Starter Kit. It should already be assembled. If not, assemble it now.
    - You will have received instructions with the kit on how to assemble it.
- The Starter Kit has a temperature/humidity sensor attached. If it does not, find the sensor and attach it.
    - The humidity sensor is a small black box attached with a thin cable. It is marked with the initials "HUM."
- Connect the Starter Kit to your computer with a USB cable. 
    - The Kit come with a USB cable, it is attached to the Hub device inside the Starter Kit.
- If you have an older version of the Phidgets, you may need to update its firmware. See the following for details:
    - https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
- As mentioned earlier, make sure you are running this notebook in the Google Chrome browser. 

## Libraries

We include a few libraries, for plotting and doing numerical work.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

## Data storage

The data has to go somewhere, so we create a few buffers to store the temperature and humidity readings.

In [ ]:
buffer_len = 100

h_buffer = np.zeros(buffer_len)
t_buffer = np.zeros(buffer_len)

## Gauges and charts

We create the two gauges for temperature and humidity, and the two charts to display the data over time. The gauges are called `t_indicator` and `h_indicator`, and the charts are called `t_chart` and `h_chart`.

We use the features of Plotly to put these four items into a single figure, using the `subplot()` function. We also add titles and some default ranges for the y-axes in the charts, to make this all work with the callback functions above. 

We then turn the figure into a live widget, using the `FigureWidget()` function.

Once these gauges and charts are all set up, we will connect them to the Phidget sensors.

In [ ]:
x = np.arange(buffer_len)

t_indicator = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [5, 35]}}
)

h_indicator = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)


t_chart = go.Scatter(x=x, y=t_buffer, mode='lines')
h_chart = go.Scatter(x=x, y=h_buffer, mode='lines')


fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],
           [{'type' : 'xy'}, {'type' : 'xy'}]])
fig.append_trace(t_indicator, row=1, col=1)
fig.append_trace(h_indicator, row=1, col=2)
fig.append_trace(t_chart, row=2, col=1)
fig.append_trace(h_chart, row=2, col=2)

fig.update_traces(name='Temperature', showlegend = True, row=2, col=1)
fig.update_traces(name='Humidity', showlegend = True, row=2, col=2)

fig.update_yaxes(title_text="Degrees C", range=[10, 30], row=2, col=1)
fig.update_yaxes(title_text="Percentage", range=[50, 70], row=2, col=2)

plots = go.FigureWidget(fig)

plots

## Utility functions

We introduce a few utility functions, which are used to set the ranges on the vertical axes of our graphs. We don't know what temperature or humidity to expect, so we set the Y range to about ±2 degrees of the current temperature value, and ±10 percentage points for the humidity. 

This will make it easier for us to see small changes in the reading, reflected in the graphs.

In [ ]:
## Utility function to bracket a value by plus/minus 2, 5 or 10

def bracket2(value):
    return (2*np.round(value/2)-2, 2*np.round(value/2)+2)

def bracket5(value):
    return (5*np.round(value/5)-5, 5*np.round(value/5)+5)

def bracket10(value):
    return (10*np.round(value/10)-10, 10*np.round(value/10)+10)

## Update data functions

Here we define the updtate functions. These functions get called whenever the Phidgets report new values for temperature and humidity. They will link the Phidget devices to the plots on screen.

We have two buffers, or arrays, to store the temperature and humidity readings over a period of time. We chose 100 points, for simplicity, but you could change this. 

The update functions do three things:
- Update the value in the temperature or humidity gauge
- Store the new value at the end of buffer, and display the 100 values in the temperature or humidity graph
- Update the range of the y-axis for the graph, so it brackets the actual reading value.

As mentioned above, we do this bracketing so the graphs look nice, and we can see small changes in tmeperature or humidity.

Feel free to experiment with better ways to display the charts. 

In [ ]:
# define update functions to connect Phidget widgets to the gauges

def updateTemperature(change):
    plots.data[0]['value'] = change.new      
    t_buffer[0:-1] = t_buffer[1:]
    t_buffer[-1] = change.new
    plots.data[2]['y'] = t_buffer
    plots.layout['yaxis']['range'] = bracket2(change.new)

def updateHumidity(change):
    plots.data[1]['value'] = change.new
    h_buffer[0:-1] = h_buffer[1:]
    h_buffer[-1] = change.new
    plots.data[3]['y'] = h_buffer
    plots.layout['yaxis2']['range'] = bracket10(change.new)

## Setup the Phidget software

As in the earlier notebooks, the main steps are thus:

1. Import some Python libraries
2. Create the Phidget Widget as a software object to connect to the devices
3. Display the widget and the gauges
4. Connect by pressing a button on screen
5. Link the Phidget data to the gauges, for live updates. View the gauges.
6. Close the sensor when you are done

## Step 1.

Let's import a few Python libraries that we need. 

The **anywidget** library is still in development, so we do a pip install if it is not already on the hub. 

In [ ]:
try:
    import anywidget
except:
    !pip install "anywidget[dev]"
    import anywidget
    
import pathlib
import traitlets

## Step 2.

We create the PhidgetFour class (software object) as an example of the **anywidget** models. The JS code for this tool is in the file **ph4auto.js** while the style file is in **ph4xx.css**. If you like, you can open this files in your browser to see what the code looks like.

The PhidgetFour object contains four Python variables (named temperature, humidity, moisture and luminance) which will contain the data from the sensors. 

In [ ]:
class PhidgetsFour(anywidget.AnyWidget):
    _esm = pathlib.Path("ph4auto.js")
    _css = pathlib.Path("ph4xx.css")
    # These 4 variables are used by the widget. Do not delete. 
    temperature = traitlets.Float(0).tag(sync=True)
    humidity = traitlets.Float(0).tag(sync=True)
    moisture = traitlets.Float(0).tag(sync=True)
    luminance = traitlets.Float(0).tag(sync=True)

pf = PhidgetsFour()

## Step 3.

We now open and display the Phidget widget along with the gauges and plots.

When you run the following cell, you should see a button displayed and some text, followed by the gauges and plots. 

In [ ]:
display(pf,plots)

## Step 4. 
Clicking on the "Connect" button above will open a window that asks you to select the Phidget Hub. Select the hub and click "Connect." The window looks like this:
<img src="images/Pconnect.png" alt="Hey"  width = 300 />

If you don't get this window, perhaps you are using the wrong browser. (Use Chrome, Chromium, or Opera).

If you get the window but don't see the Phidget device, check to be sure the Phidget VINT hub is connected to your computer with a USB cable.

### Check:
If all is well, the status text will report that the USB port is connected and the device channels are open. You should also see the value for temperature and humidity

If you connected a moisture probe and the light detector from the Phidgets plant kit, you will see these values displayed as well. 

If you don't see any values, check again that the Phidget VINT device is connected to your computer with a USB cable. Also check that you have a Phidget temperature/humidity sensor connected to your VINT with the appropriate cable. 

## Step 5.

Now we link the gauges and plots to the Phidgets. Run the following cell and you will see the gauges update as the tempature and humidity changes.

In [ ]:
pf.observe(updateTemperature, names=["temperature"])
pf.observe(updateHumidity, names=["humidity"])

## PAUSE HERE

The gauges should be running now. Everything is live.  Scroll up and check it out the gauges, to see the temperature and humitity values. The plots should be recording data as well.

Try breathing on the sensor, so you can see changes in the readings. 

We can even display a copy of the plots right here, also live. 

In [ ]:
display(plots)

## Step 6. Closing down the sensors

It is **really important** to close the sensors when you are done. Otherwise they will keep busy forever, always trying to update the gauges with the latest values. Also, no other notebook can use the Phidgets while this one is active. 

To close the Phidget, go back to the Connect button above, and click where it says "Click to disconnect."

You can also simply close this notebook. 

## In case of errors

Do you see gauges just above this cell, but nothing is happening? Go back and look for the "Connect" button in one of the cells. Click on it to get the Phidget hardware to connect to the software.

Still having problems? What kind of errors can happen? What should we do about them?

**Symptom 1**. When you click the Connect button, status text says "USB not connected"
- Problem is likely that your browser does not support WebUSB.
    - Solution is to switch to the latest versions of Chrome, Chromium, Opera or Edge.
- Another problem could be that the Phidget VINT device is not connected via a USB cable. You should see a window like this <img src="images/Pconnect.png" alt="Hey"  width = 300 /> Select the VINT device once it appears.
- Another notebook or piece of software is already connected to the Phidget device.
    - Only one notebook at a time can use the Phidget. So close down any other notebook using Phidgets, or exit any other software using the device. 
- You might have an older version of a Phidget that needs a firmware update.
    - see here for the solution: https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
  
    
**Symptom 2**. When you click the Connect button, status text is "USB connected, channel not open."
- Check to see that you have a Phidget temperature/humidity device connected to your Phidget hub or VINT device


## Going further

Some ideas to try on your own:

- Is there a better way to bracket the values when we set the y-axis limits? Can we make the display nicer?
- Can we add some "memory" to the y-axis ranges, so they don't jump around too much when the temperature or humidity reading crosses some critical points? For instance, when the temperature crosses an odd integer (e.g. value = 27), the range will jump around from (24,28) t0 (26,30), which is a rather ugly display.
- Can we update the charts only once a second? That way it is easier to see the slow changes in value.
- Can we record the x-axis as a time value, so we know at what time certain readings occur?
- Can we save the data into a file, on some regular basis? This should include data and time stamps, so we have an accurate historical record of the reading.


## Conclusion

We have shown how to display the temperature and humidity values from the Phidget device using a live gauges from the Plots library. We also plot the data as it comes in. 

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)